In [2]:
import numpy as np

poses = np.load('/home/chuhk/everybody_dance_now/datasets/test/test_poses.npy')

In [3]:
poses[0]

array([[ 91, 236],
       [113, 228],
       [102, 211],
       [122, 187],
       [148, 188],
       [124, 245],
       [157, 241],
       [184, 229],
       [169, 197],
       [209, 189],
       [248, 174],
       [174, 222],
       [214, 217],
       [247, 235],
       [ 87, 235],
       [ 90, 243],
       [ 89, 231],
       [ 97, 249]])

In [7]:
from tensorflow.keras.models import load_model


model = load_model('../everybody_dance_now/pose_estimator/pose_estimator.h5')

In [8]:
import tensorflow.keras

In [9]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
model_1 (Model)              [(1, 46, 46, 38), (1, 46, 52311446  
Total params: 52,311,446
Trainable params: 52,311,446
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import re
import math
import copy
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

import sys
import glob
import logging
import argparse

from collections import OrderedDict

from PIL import Image
from PIL import ImageOps
from tqdm import tqdm
from sklearn.impute import SimpleImputer

from skimage.measure import label
from collections import OrderedDict
from scipy.ndimage.filters import gaussian_filter

In [ ]:
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_) if s]


def make_layers(block, no_relu_layers):
    layers = []
    for layer_name, v in block.items():
        if 'pool' in layer_name:
            layer = nn.MaxPool2d(kernel_size=v[0], stride=v[1],
                                    padding=v[2])
            layers.append((layer_name, layer))

        else:
            conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1],
                               kernel_size=v[2], stride=v[3],
                               padding=v[4])
            layers.append((layer_name, conv2d))

            if layer_name not in no_relu_layers:
                layers.append(('relu_'+layer_name, nn.ReLU(inplace=True)))

        try:
            raise ValueError("LAYER:", layer_name)
        except:
            pass

    return nn.Sequential(OrderedDict(layers))



class BodyposeModel(nn.Module):
    def __init__(self):
        super(BodyposeModel, self).__init__()

        no_relu_layers = ['conv5_5_CPM_L1', 'conv5_5_CPM_L2', 'Mconv7_stage2_L1',\
                          'Mconv7_stage2_L2', 'Mconv7_stage3_L1', 'Mconv7_stage3_L2',\
                          'Mconv7_stage4_L1', 'Mconv7_stage4_L2', 'Mconv7_stage5_L1',\
                          'Mconv7_stage5_L2', 'Mconv7_stage6_L1', 'Mconv7_stage6_L1']

        blocks = {}
        block0 = OrderedDict([
            ('conv1_1', [3, 64, 3, 1, 1]),
            ('conv1_2', [64, 64, 3, 1, 1]),
            ('pool1_stage1', [2, 2, 0]),
            ('conv2_1', [64, 128, 3, 1, 1]),
            ('conv2_2', [128, 128, 3, 1, 1]),
            ('pool2_stage1', [2, 2, 0]),
            ('conv3_1', [128, 256, 3, 1, 1]),
            ('conv3_2', [256, 256, 3, 1, 1]),
            ('conv3_3', [256, 256, 3, 1, 1]),
            ('conv3_4', [256, 256, 3, 1, 1]),
            ('pool3_stage1', [2, 2, 0]),
            ('conv4_1', [256, 512, 3, 1, 1]),
            ('conv4_2', [512, 512, 3, 1, 1]),
            ('conv4_3_CPM', [512, 256, 3, 1, 1]),
            ('conv4_4_CPM', [256, 128, 3, 1, 1])
        ])

        block1_1 = OrderedDict([
            ('conv5_1_CPM_L1', [128, 128, 3, 1, 1]),
            ('conv5_2_CPM_L1', [128, 128, 3, 1, 1]),
            ('conv5_3_CPM_L1', [128, 128, 3, 1, 1]),
            ('conv5_4_CPM_L1', [128, 512, 1, 1, 0]),
            ('conv5_5_CPM_L1', [512, 38, 1, 1, 0])
        ])

        block1_2 = OrderedDict([
            ('conv5_1_CPM_L2', [128, 128, 3, 1, 1]),
            ('conv5_2_CPM_L2', [128, 128, 3, 1, 1]),
            ('conv5_3_CPM_L2', [128, 128, 3, 1, 1]),
            ('conv5_4_CPM_L2', [128, 512, 1, 1, 0]),
            ('conv5_5_CPM_L2', [512, 19, 1, 1, 0])
        ])

        blocks['block1_1'] = block1_1
        blocks['block1_2'] = block1_2

        self.model0 = make_layers(block0, no_relu_layers)

        for i in range(2, 7):
            blocks['block%d_1' % i] = OrderedDict([
                ('Mconv1_stage%d_L1' % i, [185, 128, 7, 1, 3]),
                ('Mconv2_stage%d_L1' % i, [128, 128, 7, 1, 3]),
                ('Mconv3_stage%d_L1' % i, [128, 128, 7, 1, 3]),
                ('Mconv4_stage%d_L1' % i, [128, 128, 7, 1, 3]),
                ('Mconv5_stage%d_L1' % i, [128, 128, 7, 1, 3]),
                ('Mconv6_stage%d_L1' % i, [128, 128, 1, 1, 0]),
                ('Mconv7_stage%d_L1' % i, [128, 38, 1, 1, 0])
            ])

            blocks['block%d_2' % i] = OrderedDict([
                ('Mconv1_stage%d_L2' % i, [185, 128, 7, 1, 3]),
                ('Mconv2_stage%d_L2' % i, [128, 128, 7, 1, 3]),
                ('Mconv3_stage%d_L2' % i, [128, 128, 7, 1, 3]),
                ('Mconv4_stage%d_L2' % i, [128, 128, 7, 1, 3]),
                ('Mconv5_stage%d_L2' % i, [128, 128, 7, 1, 3]),
                ('Mconv6_stage%d_L2' % i, [128, 128, 1, 1, 0]),
                ('Mconv7_stage%d_L2' % i, [128, 19, 1, 1, 0])
            ])

        for k in blocks.keys():
            blocks[k] = make_layers(blocks[k], no_relu_layers)

        self.model1_1 = blocks['block1_1']
        self.model2_1 = blocks['block2_1']
        self.model3_1 = blocks['block3_1']
        self.model4_1 = blocks['block4_1']
        self.model5_1 = blocks['block5_1']
        self.model6_1 = blocks['block6_1']

        self.model1_2 = blocks['block1_2']
        self.model2_2 = blocks['block2_2']
        self.model3_2 = blocks['block3_2']
        self.model4_2 = blocks['block4_2']
        self.model5_2 = blocks['block5_2']
        self.model6_2 = blocks['block6_2']


    def forward(self, x):
        out1 = self.model0(x)

        out1_1 = self.model1_1(out1)
        out1_2 = self.model1_2(out1)
        out2 = torch.cat([out1_1, out1_2, out1], 1)

        out2_1 = self.model2_1(out2)
        out2_2 = self.model2_2(out2)
        out3 = torch.cat([out2_1, out2_2, out1], 1)

        out3_1 = self.model3_1(out3)
        out3_2 = self.model3_2(out3)
        out4 = torch.cat([out3_1, out3_2, out1], 1)

        out4_1 = self.model4_1(out4)
        out4_2 = self.model4_2(out4)
        out5 = torch.cat([out4_1, out4_2, out1], 1)

        out5_1 = self.model5_1(out5)
        out5_2 = self.model5_2(out5)
        out6 = torch.cat([out5_1, out5_2, out1], 1)

        out6_1 = self.model6_1(out6)
        out6_2 = self.model6_2(out6)

        return out6_1, out6_2


class HandposeModel(nn.Module):
    def __init__(self):
        super(HandposeModel, self).__init__()

        no_relu_layers = ['conv6_2_CPM', 'Mconv7_stage2', 'Mconv7_stage3',\
                          'Mconv7_stage4', 'Mconv7_stage5', 'Mconv7_stage6']

        block1_0 = OrderedDict([
            ('conv1_1', [3, 64, 3, 1, 1]),
            ('conv1_2', [64, 64, 3, 1, 1]),
            ('pool1_stage1', [2, 2, 0]),
            ('conv2_1', [64, 128, 3, 1, 1]),
            ('conv2_2', [128, 128, 3, 1, 1]),
            ('pool2_stage1', [2, 2, 0]),
            ('conv3_1', [128, 256, 3, 1, 1]),
            ('conv3_2', [256, 256, 3, 1, 1]),
            ('conv3_3', [256, 256, 3, 1, 1]),
            ('conv3_4', [256, 256, 3, 1, 1]),
            ('pool3_stage1', [2, 2, 0]),
            ('conv4_1', [256, 512, 3, 1, 1]),
            ('conv4_2', [512, 512, 3, 1, 1]),
            ('conv4_3', [512, 512, 3, 1, 1]),
            ('conv4_4', [512, 512, 3, 1, 1]),
            ('conv5_1', [512, 512, 3, 1, 1]),
            ('conv5_2', [512, 512, 3, 1, 1]),
            ('conv5_3_CPM', [512, 128, 3, 1, 1])
        ])

        block1_1 = OrderedDict([
            ('conv6_1_CPM', [128, 512, 1, 1, 0]),
            ('conv6_2_CPM', [512, 22, 1, 1, 0])
        ])

        blocks = {}
        blocks['block1_0'] = block1_0
        blocks['block1_1'] = block1_1

        for i in range(2, 7):
            blocks['block%d' % i] = OrderedDict([
                ('Mconv1_stage%d' % i, [150, 128, 7, 1, 3]),
                ('Mconv2_stage%d' % i, [128, 128, 7, 1, 3]),
                ('Mconv3_stage%d' % i, [128, 128, 7, 1, 3]),
                ('Mconv4_stage%d' % i, [128, 128, 7, 1, 3]),
                ('Mconv5_stage%d' % i, [128, 128, 7, 1, 3]),
                ('Mconv6_stage%d' % i, [128, 128, 1, 1, 0]),
                ('Mconv7_stage%d' % i, [128, 22, 1, 1, 0])
            ])

        for k in blocks.keys():
            blocks[k] = make_layers(blocks[k], no_relu_layers)

        self.model1_0 = blocks['block1_0']
        self.model1_1 = blocks['block1_1']
        self.model2 = blocks['block2']
        self.model3 = blocks['block3']
        self.model4 = blocks['block4']
        self.model5 = blocks['block5']
        self.model6 = blocks['block6']


    def forward(self, x):
        out1_0 = self.model1_0(x)
        out1_1 = self.model1_1(out1_0)
        concat_stage2 = torch.cat([out1_1, out1_0], 1)
        out_stage2 = self.model2(concat_stage2)
        concat_stage3 = torch.cat([out_stage2, out1_0], 1)
        out_stage3 = self.model3(concat_stage3)
        concat_stage4 = torch.cat([out_stage3, out1_0], 1)
        out_stage4 = self.model4(concat_stage4)
        concat_stage5 = torch.cat([out_stage4, out1_0], 1)
        out_stage5 = self.model5(concat_stage5)
        concat_stage6 = torch.cat([out_stage5, out1_0], 1)
        out_stage6 = self.model6(concat_stage6)

        return out_stage6

In [ ]:
class constant_gpu:
    def __init__(self,
                 reshape_w=656,
                 reshape_h=368,
                 scale_factor=8,
                 wrist_elbow_ratio=0.33,
                 elbow_shoulder_dist_const=0.9,
                 width_const=1.5):
        self.reshape_w = torch.tensor(reshape_w, dtype=torch.long)
        self.reshape_h = torch.tensor(reshape_h, dtype=torch.long)
        self.scale_factor = torch.tensor(scale_factor, dtype=torch.long)
        self.wrist_elbow_ratio = torch.tensor(wrist_elbow_ratio)
        self.elbow_shoulder_dist_const = torch.tensor(elbow_shoulder_dist_const)
        self.width_const = torch.tensor(width_const)

        self.body_zeros = torch.zeros((8, reshape_h, reshape_w))
        self.hand_zeros = torch.zeros((21, reshape_h, reshape_w))
        

    def pendding(self, device):
        self.device = device
        self.reshape_w = self.reshape_w.to(device)
        self.reshape_h = self.reshape_h.to(device)
        self.scale_factor = self.scale_factor.to(device)
        self.wrist_elbow_ratio = self.wrist_elbow_ratio.to(device)
        self.elbow_shoulder_dist_const = self.elbow_shoulder_dist_const.to(device)
        self.width_const = self.width_const.to(device)

        self.body_zeros = self.body_zeros.to(device)
        self.hand_zeros = self.hand_zeros.to(device)



def parsing_body_heatmap(all_body_peaks, idx, heatmap, const):
    heatmap = torch.where(heatmap <= 0.1, const.body_zeros, heatmap)

    x_val, x = heatmap.max(1)[0].max(1)
    y_val, y = heatmap.max(2)[0].max(1)

    x = torch.where(x_val == 0, x_val.long(), x)
    y = torch.where(x_val == 0, y_val.long(), y)
    
    x[x_val < 0.20] = 0
    y[y_val < 0.20] = 0

    all_body_peaks[idx][:, 0] = x
    all_body_peaks[idx][:, 1] = y

    

def parsing_hand_heatmap(all_hand_peaks, idx, heatmap, const):
    c, h, w = heatmap.shape

    heatmap = torch.where(heatmap <= 0.04, const.hand_zeros[:c, :h, :w], heatmap) 
    
    x_val, x = heatmap.max(1)[0].max(1) 
    y_val, y = heatmap.max(2)[0].max(1) 

    x = torch.where(x_val == 0, x_val.long(), x) 
    y = torch.where(y_val == 0, y_val.long(), y)
    
    x[x_val < 0.30] = 0
    y[y_val < 0.30] = 0
    
    det_num = torch.nonzero(x).shape[0]
    
    if det_num < 15:
        x[x_val < 1.0] = 0
        y[y_val < 1.0] = 0

    all_hand_peaks[idx][:, 0] = x
    all_hand_peaks[idx][:, 1] = y


    
def hand_detect(body_peaks, const):
    detect_result = []
    hands = []

    x1, y1 = body_peaks[5][:2]  # l_shoulder_idx
    x2, y2 = body_peaks[6][:2]  # l_elbow_idx
    x3, y3 = body_peaks[7][:2]  # l_wrist_idx
    hands.append([x1, y1, x2, y2, x3, y3, True])

    x1, y1 = body_peaks[2][:2]  # r_shoulder_idx
    x2, y2 = body_peaks[3][:2]  # r_elbow_idx
    x3, y3 = body_peaks[4][:2]  # r_wrist_idx
    hands.append([x1, y1, x2, y2, x3, y3, False])

    for x1, y1, x2, y2, x3, y3, is_left in hands:
        x = x3 + const.wrist_elbow_ratio * (x3 - x2)
        y = y3 + const.wrist_elbow_ratio * (y3 - y2)

        wrist_elbow_dist = torch.sqrt(
            (x3 - x2).float() ** 2 + (y3 - y2).float() ** 2)
        elbow_shoulder_dist = torch.sqrt(
            (x2 - x1).float() ** 2 + (y2 - y1).float() ** 2)

        width = const.width_const * torch.max(wrist_elbow_dist.float(),
                                              const.elbow_shoulder_dist_const * elbow_shoulder_dist.float())

        x -= width / 2
        y -= width / 2

        if x < 0:
            x = 0
        if y < 0:
            y = 0

        width1 = width
        width2 = width

        if x + width > const.reshape_w.float():
            width1 = const.reshape_w.float() - x
        if y + width > const.reshape_h.float():
            width2 = const.reshape_h.float() - y

        width = torch.min(width1, width2)

        if width >= 20:
            detect_result.append([int(x), int(y), int(width), is_left])

    return detect_result



def extracting(resize_imgs, body_model, hand_model, verbose, const):
    batch_size = resize_imgs.shape[0]

    all_body_peaks = torch.zeros((batch_size, 8, 2), dtype=torch.long).to(const.device)
    all_hand_peaks = torch.zeros((batch_size, 2, 21, 2), dtype=torch.long).to(const.device)

    imp = SimpleImputer(missing_values=0, strategy='mean')

    inputs = torch.flip(resize_imgs, [1])

    with torch.no_grad():
        _, outputs = body_model(inputs)
    
    outputs = F.interpolate(outputs, scale_factor=(const.scale_factor, const.scale_factor))

    for idx, heatmap in (tqdm(enumerate(outputs)) if verbose else enumerate(outputs)):
        parsing_body_heatmap(all_body_peaks, idx, heatmap[:8], const)

    for idx, body_peaks in (tqdm(enumerate(all_body_peaks)) if verbose else enumerate(all_body_peaks)):
        hands_list = hand_detect(body_peaks, const)
        
        for x, y, w, is_left in hands_list:
            if x == 0: continue

            if is_left:
                hand_box = torch.flip(inputs[idx][:, y:y+w, x:x+w], [-1])
            else:
                hand_box = inputs[idx][:, y:y+w, x:x+w]
            
            origin_h, origin_w = hand_box.shape[1:]
            factor = const.reshape_h / origin_h
                
            hand_box = F.interpolate(torch.unsqueeze(hand_box, 0), scale_factor=(factor, factor))

            with torch.no_grad():
                peaks = hand_model(hand_box)

            peaks = F.interpolate(peaks, [origin_h, origin_w])
            peaks = torch.squeeze(peaks)
            
            parsing_hand_heatmap(all_hand_peaks[:, 0 if is_left else 1], idx, peaks[:21], const)

            hand_peaks = all_hand_peaks[idx, 0 if is_left else 1]
            if is_left:
                hand_peaks[:, 0] = torch.where(
                    hand_peaks[:, 0] == 0, hand_peaks[:, 0], -(hand_peaks[:, 0]-w)+x)
                hand_peaks[:, 1] = torch.where(
                    hand_peaks[:, 1] == 0, hand_peaks[:, 1], hand_peaks[:, 1]+y)

            else:
                hand_peaks[:, 0] = torch.where(
                    hand_peaks[:, 0] == 0, hand_peaks[:, 0], hand_peaks[:, 0]+x)
                hand_peaks[:, 1] = torch.where(
                    hand_peaks[:, 1] == 0, hand_peaks[:, 1], hand_peaks[:, 1]+y)
                
            hand_peaks = hand_peaks.cpu().detach().numpy()
            
            imp.fit(hand_peaks)
            hand_peaks = imp.transform(hand_peaks)

            if len(hand_peaks[0]) == 0: 
                continue

            all_hand_peaks[idx, 0 if is_left else 1] = torch.from_numpy(hand_peaks).to(device)

    return all_body_peaks.cpu().detach().numpy(), all_hand_peaks.cpu().detach().numpy()

In [ ]:
device = "cuda:0"

const = constant_gpu()
const.pendding(device)

body_model = BodyposeModel()
body_model.load_state_dict(torch.load('./models/body_model.pth'))
body_model = body_model.to(device)
body_model.eval()
body_model(torch.zeros((1, 3, const.reshape_w, const.reshape_h)).to(device))

hand_model = HandposeModel()
hand_model.load_state_dict(torch.load('./models/hand_model.pth'))
hand_model = hand_model.to(device)
hand_model.eval()
hand_model(torch.zeros((1, 3, const.reshape_w, const.reshape_h)).to(device))


def img_to_tensor(img, factor=256., norm=0.5, device="cpu"):
    return torch.tensor((np.array(img) / factor) - norm, dtype=torch.float32).transpose(0, 2).transpose(1, 2)


def resize_to_tensor(imgs, const):
    assert len(imgs) != 0
    imgs.sort(key=natural_key)
    return torch.cat([
        torch.unsqueeze(
            img_to_tensor(
                Image.open(file)
                .resize((const.reshape_w, const.reshape_h))), 0).to(device) for file in imgs], dim=0)

In [ ]:
def draw_bodypose(canvas, candidate):
    stickwidth = 4
    limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10], \
               [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17], \
               [1, 16], [16, 18], [3, 17], [6, 18]]

    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
              [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
              [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    
    for idx, (x, y) in enumerate(candidate[0]):
        cv2.circle(canvas, (x, y), 4, colors[idx], thickness=-1)
            
    return canvas


def draw_handpose(canvas, all_hand_peaks, show_number=False):
    edges = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8], [0, 9], [9, 10], \
             [10, 11], [11, 12], [0, 13], [13, 14], [14, 15], [15, 16], [0, 17], [17, 18], [18, 19], [19, 20]]
    fig = Figure(figsize=plt.figaspect(canvas))

    fig.subplots_adjust(0, 0, 1, 1)
    fig.subplots_adjust(bottom=0, top=1, left=0, right=1)
    bg = FigureCanvas(fig)
    ax = fig.subplots()
    ax.axis('off')
    ax.imshow(canvas)

    width, height = ax.figure.get_size_inches() * ax.figure.get_dpi()

    for peaks in all_hand_peaks:
        for ie, e in enumerate(edges):
            if np.sum(np.all(peaks[e], axis=1)==0)==0:
                x1, y1 = peaks[e[0]]
                x2, y2 = peaks[e[1]]
                ax.plot([x1, x2], [y1, y2], color=matplotlib.colors.hsv_to_rgb([ie/float(len(edges)), 1.0, 1.0]))

        for i, keyponit in enumerate(peaks):
            x, y = keyponit
            ax.plot(x, y, 'r.')
            if show_number:
                ax.text(x, y, str(i))
    bg.draw()
    canvas = np.fromstring(bg.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    return canvas


def _resize(img, dsize, fx=None, fy=None, anti_aliasing=True):
    from skimage.transform import resize
    
    if fx is None:
        return resize(img, (dsize[1], dsize[0]), anti_aliasing, preserve_range=True)
    
    else:
        return resize(img, 
                      (math.ceil(img.shape[0] * fx), 
                       math.ceil(img.shape[1] * fy)),
                      anti_aliasing, preserve_range=True)

In [ ]:
img_list = sorted(glob.glob(frame_path + '*'))
img = img_list[50:51]

tenser = resize_to_tensor(img, const)
body_key, hand_key = extracting(tenser, body_model, hand_model, verbose=True, const=const)

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

oriImg = cv2.imread(img[0])

plt.figure(figsize=(30, 30))
oriImg = _resize(oriImg, (656, 368)).astype(np.uint8)

canvas = copy.deepcopy(oriImg)

canvas = draw_bodypose(canvas, body_key)
canvas = draw_handpose(canvas, hand_key[0])

plt.figure(figsize=(30, 30))
plt.imshow(canvas[:, :, [2, 1, 0]])
plt.axis('off')
plt.show()

In [6]:
def draw_pose_(canvas, all_hand_peaks, all_body_peaks):
    
    #body
    stickwidth = 4
    limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10], \
               [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17], \
               [1, 16], [16, 18], [3, 17], [6, 18]]

    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
              [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
              [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    
    for idx, (x, y) in enumerate(all_body_peaks[0]):
        cv2.circle(canvas, (x, y), 4, colors[idx], thickness=-1)
        cv2.line(canvas, (x, y), (x+1, y+1), color = (255, 255, 255), thickness=2)
    
    
    #hand
    edges = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8], [0, 9], [9, 10], \
             [10, 11], [11, 12], [0, 13], [13, 14], [14, 15], [15, 16], [0, 17], [17, 18], [18, 19], [19, 20]]
                 
    fig = Figure(figsize=plt.figaspect(canvas))
    
    for idx, (x, y) in enumerate(all_hand_peaks[0]):
        cv2.line(canvas, (x, y), (x+1, y+1), color = (255, 255, 255), thickness=2)
                 

    fig.subplots_adjust(0, 0, 1, 1)
    fig.subplots_adjust(bottom=0, top=1, left=0, right=1)
    bg = FigureCanvas(fig)
    ax = fig.subplots()
    ax.axis('off')
    ax.imshow(canvas)

    width, height = ax.figure.get_size_inches() * ax.figure.get_dpi()

    for peaks in all_hand_peaks:
        for ie, e in enumerate(edges):
            if np.sum(np.all(peaks[e], axis=1)==0)==0:
                x1, y1 = peaks[e[0]]
                x2, y2 = peaks[e[1]]
                ax.plot([x1, x2], [y1, y2], color=matplotlib.colors.hsv_to_rgb([ie/float(len(edges)), 1.0, 1.0]))
                cv2.line(canvas, (x1, x2), (x2, y2), color = (255, 255, 255), thickness=2)

        for i, keyponit in enumerate(peaks):
            x, y = keyponit
            ax.plot(x, y, 'r.')
            if show_number:
                ax.text(x, y, str(i))
    bg.draw()
    canvas = np.fromstring(bg.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    return canvas